In [1]:
import os
import time
import cv2 as cv
from glob import glob
from scipy.io import loadmat

In [2]:
caltechDataDir = './data_and_labels/data'
caltechLabelsDir = './data_and_labels/annotations/annotations'
imageFormat = 'jpg'
genImagesDir_train = './dataset/images/train'
genImagesDir_val = './dataset/images/val'
genImagesDir_test = './dataset/images/test'
genLabelsDir_train = './dataset/labels/train'
genLabelsDir_val = './dataset/labels/val'
genLabelsDir_test = './dataset/labels/test'
frameSize = (640, 480)

## Data

In [3]:
def saveImage(frame, setName, fileName, counter):
    setId = setName[-2:]
    if int(setId) < 6:
        destFolder = genImagesDir_train
    elif int(setId) < 9:
        destFolder = genImagesDir_val
    else:
        destFolder = genImagesDir_test
    
    try:
        cv.imwrite(os.path.join(destFolder, f'{setName}_{fileName}_{counter}.{imageFormat}'), frame)
    except cv.error as openCVError:
        print(str(openCVError))

In [4]:
def imageGenerator():
    # Checking the input directory if contains .seq files
    seqFiles = sorted(glob(f'{caltechDataDir}/*/*/*/*.seq'))
    if not seqFiles:
        print(f'{caltechDataDir} contains no ".seq" files!')
        return
    
    # Creating the output folder if it doesn't exist
    #if not os.path.exists(genImagesDir):
    #    os.makedirs(genImagesDir)
    
    # Processing .seq files
    for seqFile in seqFiles:
        counter = 0

        # Grabs the parent directory of .seq file, e.g. set001
        setName = os.path.basename(os.path.dirname(seqFile))
        
        # Grabs the name of .seq file, e.g. V000
        fileName = os.path.basename(seqFile).split('.')[0]
        
        try:
            capture = cv.VideoCapture(seqFile)
            
            while True:
                # Read all frames from the sequence file
                existed, frame = capture.read()
                if not existed:
                    break
                
                # Saving image
                saveImage(frame, setName, fileName, counter)
                
                counter += 1
        except Exception as error:
            print(str(error))

In [6]:
imageGenerator()

## Labels

In [3]:
classes = ['person', 'people']  # Classes in Caltech Pedestrian dataset

In [4]:
def convertBoxFormat(box):
    (left, top, width, height) = box
    (imageWidth, imageHeight) = frameSize
    drawWidth = 1./imageWidth
    drawHeight = 1./imageHeight
    x = (left + width / 2.0) * drawWidth
    y = (top + height / 2.0) * drawHeight
    w = width * drawWidth
    h = height * drawHeight
    return (x, y, w, h)

In [5]:
def annotationGenerator():
    # Initialization
    numberOfTruthBoxes = 0
    #datasets = {
    #    'train': open(f'{genLabelsDir}/train.txt', 'w'),
    #    'val': open(f'{genLabelsDir}/val.txt', 'w'),
    #    'test': open(f'{genLabelsDir}/test.txt', 'w')
    #}
    
    # Checking the input directory if contains .vbb files
    vbbFiles = sorted(glob(f'{caltechLabelsDir}/*/*.vbb'))
    if not vbbFiles:
        print(f'{caltechLabelsDir} contains no ".vbb" files!')
        return
    
    # Processing .vbb files
    for vbbFile in vbbFiles:
        # Grabs the name of .vbb file, e.g. V000
        fileName = os.path.basename(vbbFile).split('.')[0]
        
        # Grabs the parent directory of .vbb file, e.g., set001
        setName = os.path.basename(os.path.dirname(vbbFile))
        
        setId = setName[-2:]
        if int(setId) < 6:
            destFolder = genLabelsDir_train
        elif int(setId) < 9:
            destFolder = genLabelsDir_val
        else:
            destFolder = genLabelsDir_test
        
        # Processing .vbb file
        try:
            vbb = loadmat(vbbFile)
            frameLists = vbb['A'][0][0][1][0]
            frameLabel = [str(item[0]) for item in vbb['A'][0][0][4][0]]
            
            # Processing frames
            for frameId, frameValue in enumerate(frameLists):
                labels = ''
                if len(frameValue) > 0:
                    # Processing frames with labels
                    for pedestrianId, pedestrianPos in zip(frameValue['id'][0], frameValue['pos'][0]):
                        pedestrianId = int(pedestrianId[0][0]) - 1
                        pedestrianPos = pedestrianPos[0].tolist()
                        
                        # Class filter and height filter
                        if frameLabel[pedestrianId] in classes and pedestrianPos[3] > 30 and pedestrianPos[3] <= 80:
                            classIndex = classes.index(frameLabel[pedestrianId])
                            yoloBoxFormat = convertBoxFormat(pedestrianPos)
                            labels += str(classIndex) + ' ' + ' '.join([str(item) for item in yoloBoxFormat]) + '\n'
                            numberOfTruthBoxes += 1
                    
                # Writing labels to file
                filePath = destFolder + '/' + setName + '_' + fileName + '_' + str(frameId) + '.txt'
                labelFile = open(filePath, 'w')
                labelFile.write(labels)
                labelFile.close()
        except Exception as error:
            print(str(error))

In [6]:
annotationGenerator()